In [12]:
import sys
sys.path.append('..')

import numpy as np
from scipy.stats import entropy
import torch
from seqCGAN.generator import Generator  # 假设你有一个定义好的 Discriminator 类
import json
import random
import math
import struct
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from scipy.stats import wasserstein_distance
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from dtaidistance import dtw

In [13]:
TOTAL_LEN = 114
# NPRINT_REAL_WIDTH = 50*8
NPRINT_REAL_WIDTH = 22*8
# LABEL_DICT = {'facebook': 0, 'skype': 1, 'aim': 2, 'email': 3, 'voipbuster': 4, 'hangouts': 5, 'youtube': 6, 'sftp': 7, 'icq': 8,  'ftps': 9, 'vimeo': 10, 'spotify': 11, 'netflix': 12, 'bittorrent': 13}
LABEL_DICT = {'facebook': 0, 'skype': 1}
# LABEL_DICT = {'facebook': 0, 'skype': 1, 'email': 2, 'voipbuster': 3, 'hangouts': 4, 'youtube': 5, 'ftps': 6, 'vimeo': 7, 'spotify': 8, 'netflix': 9, 'bittorrent': 10}
# LABEL_DICT = {'facebook': 0, 'skype': 1, 'email': 2, 'voipbuster': 3, 'youtube': 4, 'ftps': 5, 'vimeo': 6, 'spotify': 7, 'netflix': 8, 'bittorrent': 9}
# LABEL_DICT = {'email': 0, 'youtube': 1, 'ftps': 2, 'vimeo': 3, 'spotify': 4, 'netflix': 5, 'bittorrent': 6}

SEQ_DIM = 2
MAX_PKT_LEN = 3001
MAX_TIME = 10000
MAX_PORT = 65535
MAX_SEQ_LEN = 16
WORD_VEC_SIZE = 8
# SEQ_DIM = WORD_VEC_SIZE * 2 + 1

label_dim = len(LABEL_DICT) 
image_dim = (1, NPRINT_REAL_WIDTH, NPRINT_REAL_WIDTH)  # 生成单通道图像
noise_dim = 100  # 噪声维度
batch_size = 128
source_name = '../data/vpn_data_small.json'
bins_name = '../bins/bins_small_new.json'
model_name = '../save_seq/generator_1200.pth'

with open('../wordvec/word_vec_small.json', 'r') as f:
    wv_dict = json.load(f)
    
wv = {}
for key, metrics in wv_dict.items():
    wv[key] = torch.tensor(metrics, dtype=torch.float32)

x_list = [wv_tensor.size(0) for wv_tensor in wv.values()]
# x_list = [MAX_TIME, MAX_PKT_LEN]

In [14]:
bins_data = {}
with open(bins_name, 'r') as f_bin:
    bins_data = json.load(f_bin)
    
port_intervals = bins_data['port']['intervals']
pkt_len_intervals = bins_data['packet_len']['intervals']
time_intervals = bins_data['time']['intervals']

In [15]:
# generator = Generator(label_dim, noise_dim, SEQ_DIM, MAX_SEQ_LEN, 'cpu')
generator = Generator(label_dim,SEQ_DIM,MAX_SEQ_LEN,x_list,'cpu')

# 加载模型权重
checkpoint = torch.load(model_name)  # 加载保存的权重字典
generator.load_state_dict(checkpoint)  # 将权重字典加载到模型中

# 切换到评估模式
generator.eval()

/tmp/ipykernel_1171225/3048651150.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_name)  # 加载保存的权重字典


Generator(
  (emb): ModuleList(
    (0): Embedding(41, 128)
    (1): Embedding(369, 128)
  )
  (condition_fix): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (length_fc): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU(inplace=True)
  )
  (combine_fc): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=320, out_features=512, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (lstm): LSTM(512, 512, num_layers=4, batch_first=True)
  (output_layer): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=576, out_features=41, bias=True)
      (1): Identity()
    )
    (1): Sequential(
      (0): Linear(in_features=576, out_features=369, bias=True)
      (1): Identity()
    )
  )
  (softmax): LogSoftmax(dim=-1)
)

In [16]:
class SequenceDataset(Dataset):
    def __init__(self, datas, label_str):
        """
        :param sequences: 一个包含真实序列的列表，每个序列是一个 ndarray 或 list
        """
        self.datas = datas
        self.lengths = [len(seq) for seq in datas]  # 提取序列长度
        label_id = LABEL_DICT[label_str]
        self.label_one_hot = torch.zeros(label_dim).to('cpu') 
        self.label_one_hot[label_id] = 1

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        return self.lengths[idx],self.label_one_hot 

In [17]:
def get_real_data(file_name):
    with open(file_name,'r') as f:
        json_data = json.load(f)['data']
    data_dic = {}
    
    for item in json_data:
        label_str = item['labels'][0]
        if label_str not in data_dic:
            data_dic[label_str] = []

        packets_len = item['meta']['packets']
        im = bytes.fromhex(item['nprint'])

        line = im[0:TOTAL_LEN]
        tcp_dport = line[32:34]
        udp_dport = line[92:94]

        dport = bytearray(a | b for a, b in zip(tcp_dport, udp_dport))
        dport = int.from_bytes(dport, 'big')
    
        data_item = []
        for i in range(min(16,packets_len)):
            line = im[i*TOTAL_LEN:i*TOTAL_LEN+TOTAL_LEN]
            # time_h, time_l, pl = struct.unpack("IIh", line[:10])
            # time_l //= 1e4
            # time = time_h + time_l/100
            time_h,time_l, pkt_len = struct.unpack("IIh", line[:10])
            time_l //= 1e4
            time = time_h + time_l/100
            time = time_h % 1000 + time_l / 10
            
            data_item.append([time, pkt_len])
            
        data_dic[label_str].append(data_item)
        
    return data_dic

def get_fake_data(real_data, label_str):
    dataset = SequenceDataset(real_data,label_str)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=False)
    generated_sequences = []
    with torch.no_grad():
        for lengths, labels in dataloader:
            lengths = lengths.to(torch.device("cpu"))  # 确保在同一个设备上
            labels = labels.to(torch.device("cpu"))
            # print(lengths.shape)
            # print(labels.shape)
            batch_size = lengths.size(0)
            # 生成随机噪声向量
            # noise = torch.randn(len(lengths), noise_dim)
            # 输入生成器生成数据
            fake_data = generator.sample(batch_size,labels,lengths)
            # 将生成结果按序列长度截断
            for i, length in enumerate(lengths):
                generated_sequences.append(fake_data[i, :length].cpu().tolist())
       
    final_seqs = []         
    for seq in generated_sequences:
        f_seq = []
        for i in range(len(seq)):
            [time,pkt_len] = seq[i]       
            # time_id = math.floor((time + 1)/2*len(time_intervals[i]))
            # pkt_len_id = math.floor((pkt_len + 1)/2*len(pkt_len_intervals[i]))
            # dport_id = math.floor((dport + 1)/2*len(port_intervals))
        
            time = round(random.uniform(time_intervals[time][0], time_intervals[time][1]),2)
            pkt_len = round(random.uniform(pkt_len_intervals[pkt_len][0], pkt_len_intervals[pkt_len][1]))
            # dport = round(random.uniform(port_intervals[dport_id][0],port_intervals[dport_id][1]))
            time = time
            pkt_len = pkt_len
            f_seq.append([time,pkt_len])
        final_seqs.append(f_seq)
    return final_seqs

def pad(sequence, target_length, pad_value=np.nan):
    seq_len = len(sequence)
    if seq_len < target_length:
        padding = [[pad_value] * len(sequence[0])] * (target_length - seq_len)
        return sequence + padding  # 填充
    return sequence

def get_training_dataset(data_dic):
    x = []
    y = []
    label_count = 0
    for label, data in data_dic.items():
        x.extend(data)  # 添加所有序列
        y.extend([label_count] * len(data))
        label_count += 1
    return x,y

In [18]:
real_datas = get_real_data(source_name)

fake_datas = {}
for label, data in real_datas.items():
    fake_data = get_fake_data(data,label)
    fake_datas[label] = fake_data
    
x_real, y_real = get_training_dataset(real_datas)
x_fake, y_fake = get_training_dataset(fake_datas)
# print(x_real)
# print(y_real)
# print(x_fake)
# print(y_fake)

In [19]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# 提取统计特征（均值、方差、最大值、最小值等）
def extract_features(X):
    features = []
    for seq in X:
        seq = np.array(seq)
        mean_feat = np.mean(seq, axis=0)
        std_feat = np.std(seq, axis=0)
        max_feat = np.max(seq, axis=0)
        min_feat = np.min(seq, axis=0)
        first_feat = seq[0]
        length = len(seq)
        features.append(np.hstack([mean_feat, std_feat, max_feat, min_feat, first_feat, length]))
    return np.array(features)

def test_xgboost(X_train, Y_train, X_test, Y_test):
    X_train_features = extract_features(X_train)
    X_test_features = extract_features(X_test)
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train_features, Y_train)
    Y_pred = model.predict(X_test_features)
    accuracy = accuracy_score(Y_test, Y_pred)
    return accuracy

In [20]:
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

class TrafficDataset(Dataset):
    def __init__(self, X, Y):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in X]
        self.Y = torch.tensor(Y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    
def collate_fn(batch):
    X, Y = zip(*batch)
    X_padded = pad_sequence(X, batch_first=True, padding_value=0)  # 填充变长序列
    lengths = torch.tensor([len(x) for x in X])  # 记录原始长度
    return X_padded, lengths, torch.tensor(Y)

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, device):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.device = device

    def forward(self, x, lengths):
        packed_x = nn.utils.rnn.pack_padded_sequence(x, lengths.to('cpu'), batch_first=True, enforce_sorted=False).to(self.device)
        _, (hidden, _) = self.lstm(packed_x)
        return self.fc(hidden[-1])  # 取最终隐藏状态
    
def test_lstm(X_train, Y_train, X_test, Y_test):
    train_dataset = TrafficDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, collate_fn=collate_fn, shuffle=True)
    test_dataset = TrafficDataset(X_test, Y_test)
    test_loader = DataLoader(test_dataset, batch_size=128, collate_fn=collate_fn)
    
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(device)
    model = LSTMClassifier(SEQ_DIM, 128, label_dim, device).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(100):
        model.train()
        for X, lengths, Y in train_loader:
            X, Y, lengths = X.to(device), Y.to(device), lengths.to(device)
            Y_pred = model(X, lengths)
            loss = criterion(Y_pred, Y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X, lengths, Y in test_loader:
                X, Y, lengths = X.to(device), Y.to(device), lengths.to(device)
                Y_pred = model(X, lengths)
                _, predicted = torch.max(Y_pred, 1)
                total += Y.to('cpu').size(0)
                correct += (predicted == Y).sum().item()
        accuracy = correct / total
    return accuracy

In [21]:
class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, device):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)  # 全局池化
        self.fc = nn.Linear(128, num_classes)
        self.device = device

    def forward(self, x):
        x = x.permute(0, 2, 1)  # 交换维度以适应 Conv1d (batch, feature_dim, seq_len)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        return self.fc(x)
    
def test_cnn(X_train, Y_train, X_test, Y_test):
    train_dataset = TrafficDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, collate_fn=collate_fn, shuffle=True)
    test_dataset = TrafficDataset(X_test, Y_test)
    test_loader = DataLoader(test_dataset, batch_size=128, collate_fn=collate_fn)
    
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(device)
    model = CNNClassifier(SEQ_DIM, label_dim, device).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(100):
        model.train()
        for X, lengths, Y in train_loader:
            X, Y, lengths = X.to(device), Y.to(device), lengths.to(device)
            Y_pred = model(X)
            loss = criterion(Y_pred, Y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X, lengths, Y in test_loader:
                X, Y, lengths = X.to(device), Y.to(device), lengths.to(device)
                Y_pred = model(X)
                _, predicted = torch.max(Y_pred, 1)
                total += Y.to('cpu').size(0)
                correct += (predicted == Y).sum().item()
        accuracy = correct / total
    return accuracy
    

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(x_real, y_real, test_size=0.2, random_state=42)
X_train_fake, X_test_fake, Y_train_fake, Y_test_fake = train_test_split(x_fake, y_fake, test_size=0.2, random_state=42)

methods = [test_xgboost, test_cnn, test_lstm]

for func in methods:
    accuracy_real = func(X_train, Y_train, X_test, Y_test)
    accuracy_fake = func(X_train_fake, Y_train_fake, X_test, Y_test)
    accuracy_mix = func(X_train_fake + X_train, Y_train_fake + Y_train, X_test, Y_test)
    print(f'Accuracy on real data of {func.__name__}:', accuracy_real)
    print(f'Accuracy on fake data of {func.__name__}:', accuracy_fake)
    print(f'Accuracy on mixed data of {func.__name__}:', accuracy_mix)


/home/xiaozj/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:47:23] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/xiaozj/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:47:23] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/xiaozj/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:47:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy on real data of test_xgboost: 0.930715935334873
Accuracy on fake data of test_xgboost: 0.7124711316397229
Accuracy on mixed data of test_xgboost: 0.9318706697459584
Accuracy on real data of test_cnn: 0.8833718244803695
Accuracy on fake data of test_cnn: 0.8060046189376443
Accuracy on mixed data of test_cnn: 0.8625866050808314
Accuracy on real data of test_lstm: 0.8960739030023095
Accuracy on fake data of test_lstm: 0.8279445727482679
Accuracy on mixed data of test_lstm: 0.894919168591224
